# Real EEG data example

In [ ]:
%matplotlib widget
import os
import numpy as np
from scipy import signal, fft
from pathlib import Path

from matplotlib import pyplot
from lab1_utils import *


pyplot.rcParams.update({
    "text.usetex": True,
    "font.family": "Helvetica",
    "figure.constrained_layout.use": True,
    "savefig.dpi": 300
})
# rng = np.random.default_rng(12345)

In [ ]:
with np.load('propofol_eeg_demo.npz') as f:
    data = f['data']
    times = f['times']

baseline        = data[0, np.logical_and(times >= 500., times < 510.)] 
deep_anesthesia = data[0, np.logical_and(times >= 4705., times < 4715.)]

fs = 1 / (times[1] - times[0])
fs

In [ ]:
fig, ax = pyplot.subplots(figsize=(8, 2))
ax.plot(np.arange(baseline.shape[-1]) / 100, baseline.T * 1e6, linewidth=0.5)
ax.set_ylim([-50, 50])
ax.set_title('Baseline')
ax.set_ylabel('amplitude (µV)')
ax.set_xlabel('time (s)')

In [ ]:
S_yy_mt_est, freqs = multitaper_periodogram(baseline, fs=fs)
fig, ax = pyplot.subplots(figsize = (6,3))
ax.plot(freqs, 10*np.log10(S_yy_mt_est) + 120, linewidth=0.5)
ax.set_xlim([0, fs/2])
ax.set_title('Baseline Spectrum')
ax.set_ylabel('PSD (dB)')
ax.set_xlabel('Freq (Hz)')

In [ ]:
fig, ax = pyplot.subplots(figsize=(8, 2))
ax.plot(np.arange(deep_anesthesia.shape[-1]) / 100, deep_anesthesia.T * 1e6, linewidth=0.5)
ax.set_ylim([-50, 50])
ax.set_title('Deep Anesthesia')
ax.set_ylabel('Amplitude (µV)')
ax.set_xlabel('Time (s)')

In [ ]:
S_yy_mt_est, freqs = multitaper_periodogram(deep_anesthesia, fs=fs, ntapers=7)
fig, ax = pyplot.subplots(figsize = (6,3))
ax.plot(freqs, 10*np.log10(S_yy_mt_est) + 120, linewidth=0.5)
ax.set_xlim([0, fs/2])
ax.set_ylabel('PSD (dB)')
ax.set_xlabel('Freq (Hz)')
ax.set_title('Deep Anesthesia Spectrum')

# Multitaper spectrogram function

Create a function `multitaper_spectrogram` that takes in a window size, sampling frequency, time-half-bandwidth product, and number of tapers to create a spectrogram using your `multitaper_periodogram` function

In [ ]:
window_duration = 4
nw = 4
ntapers = 7

powers, freqs, times = multitaper_spectrogram(data[0,:], window_size=int(window_duration * fs), fs=fs, nw=nw, ntapers=ntapers)

In [ ]:
fix, ax = pyplot.subplots(figsize=(8,4))
ax.pcolormesh(times, freqs, 10*np.log10(powers)+120, cmap='jet', clim=[-10, 20])
ax.set_ylim([0, fs/2])
ax.set_ylabel('Frequency (Hz)')
ax.set_xlabel('Time (s)')
ax.set_title('Anesthesia case')